---
# **RUN on dataset**
---

In [ ]:
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.analysis.stats import StatAnalysis
from pyVHR.plot.visualize import *
import os
import pandas as pd
import plotly.express as px

# main functions 

def ANALYSIS(dataset, approach, metrics=['MAE'], j_data=False, writeFile=False):
  """Print boxplot analysis"""

  cd = os.getcwd() # current dir
  if not j_data:
    print('## Dataset: ' + dataset)
    print('   Approach: ' + approach)
    path = path_results + dataset + "_" + approach + ".h5"
    filenameH5 = os.path.join(cd, path)
    print('   h5      :', path)
    cfg = path_results + dataset + "_" + approach + ".cfg"
    print('   cfg     :', cfg, end='\n\n')
  else:
    print('## Path dir: ' + dataset)
    print('   Approach: ' + approach)
    path = path_results + dataset 
    filenameH5 = os.path.join(cd, path)
    print('   path  = ', path)

  # boxplot
  for m in metrics:
    st = StatAnalysis(filenameH5, join_data=j_data)
    if m=='MAE': 
      scale = 'log' 
    else: 
      scale = 'linear'
    fig = st.displayBoxPlot(metric=m, scale=scale)
    fig.show(renderer = "colab")
    if writeFile:
      filenamePNG = path_results + dataset + "_" + approach + "_" + m + "_MAE.png"
      fig.write_image(filenamePNG)  

def STATS(dataset, approach='clustering'):
  """Do stats analysis"""
  
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + approach + ".h5"
  df = pd.read_hdf(os.path.join(cd, path))
  MAE_mean, MAE_std, MAE_median = np.round(df['MAE'].mean()[0],2),     np.round(df['MAE'].std(),2),  np.round(df['MAE'].median(),2)
  RMSE_mean, RMSE_std, RMSE_median = np.round(df['RMSE'].mean()[0],2), np.round(df['RMSE'].std(),2), np.round(df['RMSE'].median(),2)
  PCC_mean, PCC_std, PCC_median = np.round(df['PCC'].mean()[0],2),     np.round(df['PCC'].std(),2),  np.round(df['PCC'].median(),2)
  CCC_mean, CCC_std, CCC_median = np.round(df['CCC'].mean()[0],2),     np.round(df['CCC'].std(),2),  np.round(df['CCC'].median(),2)
  SNR_mean, SNR_std, SNR_median = np.round(df['SNR'].mean()[0],2),     np.round(df['SNR'].std(),2),  np.round(df['SNR'].median(),2)
  MAX_mean, MAX_std, MAX_median = np.round(df['MAX'].mean()[0],2),     np.round(df['MAX'].std(),2),  np.round(df['MAX'].median(),2)

  table = [[MAE_mean, MAE_std, MAE_median], [RMSE_mean, RMSE_std, RMSE_median], [PCC_mean, PCC_std, PCC_median],
           [CCC_mean, CCC_std, CCC_median], [SNR_mean, SNR_std, SNR_median], [MAX_mean, MAX_std, MAX_median]]
  stats = pd.DataFrame(table, columns = [ 'MEANS', 'STD', 'MEDIANS'], index=['MAE', 'RMSE', 'PCC', 'CCC', 'SNR', 'MAX'])

  print('\n** Stats table **')
  print(stats)
					
def RUN(dataset, method):
  """Run methods on a dataset"""

  print('## Dataset: ' + dataset)
  print('Using...')
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + method + ".h5"
  print('   filename h5: ', path)
  cfg = path_results + dataset + "_" + method + ".cfg"
  print('   cfg        :', cfg)
  print('\n')

  # pipeline
  pl = Pipeline()
  res = pl.run_on_dataset(os.path.join(cd, cfg), verb=1)
  res.saveResults(filenameH5)
  print('Written file: ' + filenameH5 + '\n\n')

def DETAILS(dataset, method, metric='MAE'):
  """Show performance details on each video of the dataset"""

  #filenameH5 = path_results + dataset + "_" + method + ".h5"
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + method + ".h5"
  print('## Dataset: ' + dataset)
  print('   filename h5: ', path)
  df = pd.read_hdf(os.path.join(cd, path))
  datasetname = df['dataset'][0].upper()
  mets = set(list(df['method']))
  for m in mets:
    print('   method     :', m)
    vals = df[df['method'] == m][metric]
    fvideos = df[df['method'] == m]['videoFilename']
    x = []
    y = []
    for i in vals.index:
      n = fvideos[i]
      n = n[n.rfind('/', 0, n.rfind('/'))+1:n.rfind('/')]
      x.append(n)
      y.append(vals[i][0])
    fig = px.bar(x=x, y=y, title='dataset: ' + datasetname + ' method: ' + m)
    fig.update_xaxes(type='category')
    fig.show(renderer = "colab")


This notebook shows the complete **pipeline** that takes a **dataset** as input and computes an **estimation** of **BPMs** using a prespecified set of **rPPG** **methods**.

Below are the possible choices for the methods.
* **Methods**: `cpu_CHROM`, `cupy_CHROM`, `torch_CHROM`, `cpu_LGI`, `cpu_POS`, `cupy_POS`, `cpu_PBV`, `cpu_PCA`, `cpu_GREEN`, `cpu_OMIT`, `cpu_ICA`, `cpu_SSR`

Below are the possible choices for the datasets.
* **Datasets**: `PURE`, `UBFC1`, `UBFC2`, `ECG_Fitness_01-1`, `ECG_Fitness_01-2`... `ECG_Fitness_06-1`, `ECG_Fitness_06-2`

#Runs

In [ ]:
# run on dataset

path_results = "results/cfg/" # general path for cfg

#dataset = 'PURE'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'UBFC1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'UBFC2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'LGI_PPGI'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_01-1'   FIN
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_01-2'   
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_02-1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_02-2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_03-1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_03-2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_04-1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_04-2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_05-1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_05-2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_06-1'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

#dataset = 'ECG_Fitness_06-2'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

# Analisys

In [ ]:
## -- Holistic

path_results = "results/h5/" # general path for cfg# general path for data

dataset = 'UBFC2'
ANALYSIS(dataset, 'holistic', metrics=['MAE'])
STATS(dataset, approach='holistic')

In [ ]:
## -- Median
ANALYSIS(dataset, 'median', metrics=['MAE'])
STATS(dataset, approach='median')

In [ ]:
## -- Clustering
ANALYSIS(dataset, 'clustering', metrics=['MAE'])
STATS(dataset, approach='clustering')

In [ ]:
## -- Deep 
ANALYSIS(dataset, 'deep', metrics=['MAE'])
STATS(dataset, approach='deep')

#Details

In [ ]:
# Analysis - holistic

method = 'holistic'
DETAILS(dataset, method, metric='MAE')

In [ ]:
# Analysis - median
metric='MAE'
method = 'median'
DETAILS(dataset, method, metric)

In [ ]:
# Analysis - clustering
metric='MAE'
method = 'clustering'
DETAILS(dataset, method, metric)

# Multidataset

In [ ]:
# multi dataset
#path = path_results + 'h5_collection'
#st = StatAnalysis(path, join_data=True)
#fig = st.displayBoxPlot(metric='PCC')
#fig.show(renderer = "colab")


# Save in HTML format

In [ ]:
!jupyter nbconvert --to html --template lab run_on_dataset.ipynb